In [1]:
#model 1 (Boosting)
import pandas as pd
import numpy as np
df=pd.read_csv("parkinsons.data")
df.head(5)

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [16]:
df.dropna()
dependent=df["status"]
independent=df.loc[:, df.columns != "status"]
independent=independent.loc[:, independent.columns != "name"]

In [17]:
from sklearn.preprocessing import MinMaxScaler
Scaler=MinMaxScaler((-1,1))
x=Scaler.fit_transform(independent)
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(x,dependent,test_size=0.2)



In [18]:


from xgboost import XGBClassifier
model=XGBClassifier().fit(train_x,train_y)
pred_y=model.predict(test_x)
from sklearn.metrics import accuracy_score
print("accuracy of model=",accuracy_score(test_y,pred_y))

accuracy of model= 0.8974358974358975


In [19]:


#model 2
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
norm_x=preprocessing.StandardScaler().fit(independent).transform(independent)
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(norm_x,dependent,test_size=0.2)
neigh = KNeighborsClassifier(n_neighbors = 2).fit(train_x,train_y)
y_pred=neigh.predict(test_x)
from sklearn.metrics import accuracy_score
print("accuracy of model=",accuracy_score(test_y,y_pred))





accuracy of model= 0.8717948717948718


In [2]:
import sys
!{sys.executable} -m pip install xgboost

In [20]:
model #3 (Bagging)
df=pd.read_csv("parkinsons.data")
features=df.loc[:, df.columns != "status"]
features=features.loc[:,features.columns != "name"]
label=df["status"]


In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(features,label,test_size=0.2,random_state=7)


In [22]:
def Rfs(nest,md):
    rf=RandomForestClassifier(n_estimators=nest,max_depth=md,n_jobs=-1)
    model=rf.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    precision,recall,fscore,support=score(y_test,y_pred,pos_label=1,average="binary")
    print('EST {} / DEPTH {} ------- precision {} / recall {} / Accuracy {}'.format(nest,md,round(precision,3),round(recall,3),round((y_pred==y_test).sum()/len(y_pred),3)))
for nest in [10,50,100,150,200]:
    for md in [10,20,30,50,None]:
        Rfs(nest,md)
    

EST 10 / DEPTH 10 ------- precision 0.912 / recall 0.969 / Accuracy 0.897
EST 10 / DEPTH 20 ------- precision 0.939 / recall 0.969 / Accuracy 0.923
EST 10 / DEPTH 30 ------- precision 0.912 / recall 0.969 / Accuracy 0.897
EST 10 / DEPTH 50 ------- precision 0.938 / recall 0.938 / Accuracy 0.897
EST 10 / DEPTH None ------- precision 0.909 / recall 0.938 / Accuracy 0.872
EST 50 / DEPTH 10 ------- precision 0.886 / recall 0.969 / Accuracy 0.872
EST 50 / DEPTH 20 ------- precision 0.914 / recall 1.0 / Accuracy 0.923
EST 50 / DEPTH 30 ------- precision 0.914 / recall 1.0 / Accuracy 0.923
EST 50 / DEPTH 50 ------- precision 0.914 / recall 1.0 / Accuracy 0.923
EST 50 / DEPTH None ------- precision 0.912 / recall 0.969 / Accuracy 0.897
EST 100 / DEPTH 10 ------- precision 0.914 / recall 1.0 / Accuracy 0.923
EST 100 / DEPTH 20 ------- precision 0.914 / recall 1.0 / Accuracy 0.923
EST 100 / DEPTH 30 ------- precision 0.912 / recall 0.969 / Accuracy 0.897
EST 100 / DEPTH 50 ------- precision 0.91

In [24]:
from sklearn.ensemble import GradientBoostingClassifier

In [26]:
def Gbc(nest,md):
    gb=GradientBoostingClassifier(n_estimators=nest,max_depth=md)
    model=gb.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    precision,recall,fscore,support=score(y_test,y_pred,pos_label=1,average="binary")
    print('EST {} / DEPTH {} ------- precision {} / recall {} / Accuracy {}'.format(nest,md,round(precision,3),round(recall,3),round((y_pred==y_test).sum()/len(y_pred),3)))
for nest in [50,100,150,200,250]:
    for md in [2,3,5,7,10]:
        Gbc(nest,md)

EST 50 / DEPTH 2 ------- precision 0.939 / recall 0.969 / Accuracy 0.923
EST 50 / DEPTH 3 ------- precision 0.939 / recall 0.969 / Accuracy 0.923
EST 50 / DEPTH 5 ------- precision 0.939 / recall 0.969 / Accuracy 0.923
EST 50 / DEPTH 7 ------- precision 0.967 / recall 0.906 / Accuracy 0.897
EST 50 / DEPTH 10 ------- precision 0.966 / recall 0.875 / Accuracy 0.872
EST 100 / DEPTH 2 ------- precision 0.969 / recall 0.969 / Accuracy 0.949
EST 100 / DEPTH 3 ------- precision 0.912 / recall 0.969 / Accuracy 0.897
EST 100 / DEPTH 5 ------- precision 0.939 / recall 0.969 / Accuracy 0.923
EST 100 / DEPTH 7 ------- precision 0.967 / recall 0.906 / Accuracy 0.897
EST 100 / DEPTH 10 ------- precision 0.966 / recall 0.875 / Accuracy 0.872
EST 150 / DEPTH 2 ------- precision 0.97 / recall 1.0 / Accuracy 0.974
EST 150 / DEPTH 3 ------- precision 0.912 / recall 0.969 / Accuracy 0.897
EST 150 / DEPTH 5 ------- precision 0.939 / recall 0.969 / Accuracy 0.923
EST 150 / DEPTH 7 ------- precision 0.967 / 